# 랠릿 전처리

In [1]:
import pandas as pd
import numpy as np 
import re

In [2]:
ral_df = pd.read_csv('rallit_pre0.csv', index_col=0)  # 기존
incru_df = pd.read_csv('incru_pre0.csv', index_col=0)  # 인크루트 전처리된 파일
cat_df = pd.read_csv('cat_pre0.csv', index_col=0)  # 캐치 전처리된 파일

# 랠릿 전처리
- 주소
- 직급

In [3]:
# 주소 전처리
address_list = []

for ad in ral_df['주소']:
    try:
        # 우편번호 패턴을 찾아서 제거
        ad = re.sub(r'^\d{5}\s', '', ad).strip()
        # 글자 변경
        if '대한민국' in ad:
            ad = re.sub('대한민국', '', ad).strip()
        elif '서울' in ad and '특별시' not in ad:
            ad = re.sub('서울','서울특별시',ad).strip()
        elif '경기' in ad and '경기도' not in ad:
            ad = re.sub('경기','경기도',ad).strip()
        elif '인천' in ad and '광역시' not in ad:
            ad = re.sub('인천','인천광역시',ad).strip()

        address_list.append(ad)
    except:
        address_list.append('')
# 구,시,도로 끝나는 단어 리스트 정렬
add_list = [' '.join(re.findall(r'\b(\w+[구시도])\b', address)) for address in address_list]

ral_df['주소'] = add_list

In [4]:
# 직급 전처리
career_list = []

for car in ral_df['직급']:
    try:
        if '주니어' in car or '미들' in car or '시니어' in car:  # 무관이 없는 경우 최소단위
            career_list.append(re.findall('\d+', car)[0])

        elif '인턴' in car:  # 신입인 경우 0
            career_list.append(0)
                            
        else:  # 무관이 있는 경우 0
            career_list.append(0) 
    except:
        career_list.append(0)
ral_df['직급'] = career_list
ral_df['직급'] = ral_df['직급'].astype(int)  # int타입 변경

In [5]:
# 인덱스 재설정
ral_df.reset_index(drop=True, inplace=True)
# 전처리된 파일 저장
ral_df.to_csv('./ral_pre0.csv')

# 기존 데이터에 전처리된 파일들 머지하기
- 머지 순서는 위에서 정리한 1번부터 5번까지 진행됩니다.

In [6]:
# 중복값 제거
incru_df.drop_duplicates(inplace=True)
cat_df.drop_duplicates(inplace=True)
# 인덱스 재설정
incru_df.reset_index(drop=True, inplace=True)
cat_df.reset_index(drop=True, inplace=True)

In [7]:
# 랠릿에 인크루트 머지
df_merge1 = pd.merge(ral_df, incru_df, on='기업명', how='left', suffixes=('', '_incru'))

# 합친 데이터프레임에서 인크루트에서 온 컬럼만 선택하기
cols_to_update = [col for col in df_merge1.columns if '_incru' in col]

# 원래 컬럼의 결측치를 인크루트에서 가져온 값으로 채우기
for col in cols_to_update:
    orig_col = col.replace('_incru', '')  # 원래 컬럼 이름
    # 원래 컬럼의 결측치를 채움
    df_merge1[orig_col] = df_merge1[orig_col].combine_first(df_merge1[col])

# 추가된 컬럼 제거
df_merge1.drop(columns=cols_to_update, inplace=True)

In [8]:
# 캐치 머지
df_merge2 = pd.merge(df_merge1, cat_df, on='기업명', how='left', suffixes=('', '_catch'))

# 합친 데이터프레임에서 캐치에서 온 컬럼만 선택하기
cols_to_update = [col for col in df_merge2.columns if '_catch' in col]

# 원래 컬럼의 결측치를 캐치에서 가져온 값으로 채우기
for col in cols_to_update:
    orig_col = col.replace('_catch', '')  # 원래 컬럼 이름
    # 원래 컬럼의 결측치를 채움
    df_merge2[orig_col] = df_merge2[orig_col].combine_first(df_merge2[col])

# 추가된 컬럼 제거
df_merge2.drop(columns=cols_to_update, inplace=True)

In [9]:
# 중복된 값 제거 및 index 재설정
df_merge2 = df_merge2.drop_duplicates().reset_index(drop=True)
# 컬럼 순서 지정
df_merge2 = df_merge2[['기업명', '업종분류', '사원수', '설립연도', '기업형태', '매출액', '주소', '직무', '이용하는 기술스택',
            '자격요건', '직급', '우대사항', '해당 페이지 URL', '기업 홈페이지 URL']]

# 최종 파일 저장
df_merge2.to_csv('./rallit_end.csv')
df_merge2.to_excel('./rallit_end.xlsx')

# 직무 전처리

In [10]:
import pandas as pd
ral_jd = pd.read_csv('rallit_jd.csv')

In [11]:
ral_jd['직무']

0      데브옵스 엔지니어
1        백엔드 개발자
2        백엔드 개발자
3        백엔드 개발자
4     미디어 서버 개발자
         ...    
61     프론트엔드 개발자
62       백엔드 개발자
63       백엔드 개발자
64     프론트엔드 개발자
65     데브옵스 엔지니어
Name: 직무, Length: 66, dtype: object

In [12]:
# 기존의 직무를 제외한 새로운 데이터 생성
ral_copy = ral_df.drop(['직무'], axis=1).copy()

In [13]:
ral_jd = ral_jd.drop_duplicates(subset='기업명', keep='first')  # 변경할 직무가 있는 데이터 중복 제거
df_merged = pd.merge(ral_copy, ral_jd, on='기업명', how='left')  # 기업명을 기준으로 머지